In [1]:
import os
import cv2
import numpy as np

In [2]:
data = []
label = []
folder_dir = '/content/drive/MyDrive/Colab Notebooks/object detection/flowers'
for folder in os.listdir(folder_dir):
  for file in os.listdir(os.path.join(folder_dir, folder)):
    if file.endswith('jpg'):
      label.append(folder)
      im = cv2.imread(os.path.join(folder_dir, folder, file))
      im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
      im = cv2.resize(im_rgb, (128, 128))
      data.append(im)
    else:
      continue

In [3]:
data = np.array(data)
label = np.array(label)

In [4]:
print(label, data)

['daisy' 'daisy' 'daisy' ... 'rose' 'rose' 'rose'] [[[[ 14  13  18]
   [ 14  14  16]
   [ 13  13  14]
   ...
   [ 14  14  16]
   [ 13  13  15]
   [ 15  15  17]]

  [[ 12  12  15]
   [ 10  10  10]
   [ 11  11  11]
   ...
   [ 11  11  14]
   [  9   9  11]
   [ 12  12  14]]

  [[ 13  13  14]
   [  5   5   3]
   [  3   3   1]
   ...
   [  0   0   2]
   [  2   2   4]
   [ 14  14  16]]

  ...

  [[  9   9  11]
   [  1   1   2]
   [  1   1   2]
   ...
   [  6   4   7]
   [  9   8  10]
   [ 14  14  16]]

  [[ 13  13  15]
   [ 11  11  13]
   [ 10  10  12]
   ...
   [ 11  10  12]
   [ 10  10  12]
   [ 13  13  15]]

  [[ 14  14  16]
   [ 16  16  18]
   [ 14  14  16]
   ...
   [ 14  15  17]
   [ 13  13  15]
   [ 14  14  16]]]


 [[[  8  12  13]
   [ 10  11  15]
   [  8   9  11]
   ...
   [ 79  75  52]
   [ 74  76  51]
   [ 70  73  46]]

  [[  4  10  10]
   [  6  10  13]
   [  5   9  11]
   ...
   [ 74  75  47]
   [ 78  76  46]
   [ 69  72  44]]

  [[  5  14  12]
   [  7  12  14]
   [  7  12  13]
 

In [5]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [6]:
encoder = LabelEncoder()
y = encoder.fit_transform(label)
y = to_categorical(y, 5)
y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [7]:
x = data/255

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [9]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [11]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(x_train)

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,
                    epochs=60,
                    validation_data=(x_test, y_test),
                    verbose=1)

Epoch 1/60
95/95 [==============================] - 16s 157ms/step - loss: 0.7733 - accuracy: 0.7070 - val_loss: 0.7975 - val_accuracy: 0.6875
Epoch 2/60
95/95 [==============================] - 14s 143ms/step - loss: 0.6160 - accuracy: 0.7770 - val_loss: 0.7579 - val_accuracy: 0.7222
Epoch 3/60
95/95 [==============================] - 14s 143ms/step - loss: 0.3845 - accuracy: 0.8637 - val_loss: 0.8870 - val_accuracy: 0.7184
Epoch 4/60
95/95 [==============================] - 14s 143ms/step - loss: 0.1703 - accuracy: 0.9464 - val_loss: 1.1351 - val_accuracy: 0.7253
Epoch 5/60
95/95 [==============================] - 14s 143ms/step - loss: 0.1013 - accuracy: 0.9773 - val_loss: 1.8115 - val_accuracy: 0.6937
Epoch 6/60
95/95 [==============================] - 14s 143ms/step - loss: 0.0973 - accuracy: 0.9721 - val_loss: 1.6288 - val_accuracy: 0.7014
Epoch 7/60
95/95 [==============================] - 14s 143ms/step - loss: 0.0557 - accuracy: 0.9825 - val_loss: 1.7974 - val_accuracy: 0.6968

In [18]:
import matplotlib.pyplot as plt

categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6,6, figsize=(25, 40))

for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(x_test))
        if(categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(x_test)[k])]):
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(x_test)[k])], color='green')
            ax[i,j].imshow(np.array(x_test)[k].reshape(128, 128, 3), cmap='gray')
        else:
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(x_test)[k])], color='red')
            ax[i,j].imshow(np.array(x_test)[k].reshape(128, 128, 3), cmap='gray')

Output hidden; open in https://colab.research.google.com to view.